# Are we just in the perturbative regime?

In the 2009 paper [Pairwise Maximum Entropy Models for Studying Large Biological Systems: When They Can Work and When They Can't](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000380), Roudi et al. suggest that when we are in the perturbative regime, characterised by a small mean probability of observing neurons spike and small number of neurons $N$, the pairwise maxent model can appear to be a good model for a distribution. However, we cannot extrapolate the behaviour of the pairwise model to larger $N$, and predict that it will remain a good fit outside of the perturbative regime. In short:

> The distance between the pairwise model and **any** probability distribution appears linear in $N\bar{v}\delta t$ in the perturbative regime 

We try and investigate these claims computationally.

The perturbative regime is defined as $N\bar{v}\delta t \ll 1$, where $\bar{v}$ is the mean firing rate and $\delta t$ is the size of the time bin. As shorthand, we define $\delta \doteq \bar{v}\delta t$.  For sufficiently small time bins where we observe at most one spike within each bin, and we can identify $\delta$ with the mean probability of observing a neuron fire. 

As an example, for $N=5$ neurons, we should be in the perturbative regime with $\bar{p}(r_i=1)=\delta \ll 0.2$.

In [31]:
import numpy as np
from NumericIsing import Ising
from ThreeWise import ThreeWise

We try and reproduce the main computational results in the paper. These results show close agreement between the true distance measure between the 'pairwise' and a '3-wise distribution', and an estimate of the distance measure between these distributions, whilst in the perturbative regime. 

First, we create a '3-wise distribution'. More accurately, we define the interactions in a model that includes up to 3-wise interactions, which creates a distribution that cannot be explained by a model that includes only up to pairwise interactions. 

This distribution takes the form:
$$
    P^{(3)}(\{r_i\}) = \frac{1}{Z} \exp \left[ \sum_i h_i r + \sum_{i< j} J_{ij}r_i r_j  + \sum_{i<j<k} K_{ijk}r_i r_j r_k  \right]
$$
From the paper, the interaction terms were determined as:
- $h_i = - \ln (1/r_i^* - 1)$, where each $r_i^*$ is randomly drawn from an exponential distribution with mean 0.02
- $J_{ij}$ is drawn from a Gaussian distribution with mean 0.05 and s.d. 0.8
- $K_{ijk}$ is drawn from a Gaussian distribution with mean 0.02 and s.d. 0.5

In [32]:
def sim_p_true(N):
    """
    Function that creates a distribution that includes up to 3-wise interactions,
    by randomly sampling the interaction weights from the distributions specified in the paper
    """
    h = np.zeros(N)
    for i in range(N):
        h[i] = -np.log(1/np.random.exponential(0.02) - 1)
        
    J = np.zeros((N,N))
    for i in range(N-1):
        for j in range(i+1,N):
            J[i,j] = np.random.normal(0.05, 0.8)
            
    K = np.zeros((N,N,N))
    for i in range(N-2):
        for j in range(i+1,N-1):
            for k in range(j+1,N):
                K[i,j,k] = np.random.normal(0.02, 0.5)
    
    p_true = ThreeWise(N, h, J, K)
    return p_true

In [33]:
p_true = sim_p_true(4)
#calc 3 wise correlations
for i in range(p_true.N-2):
    for j in range(i+1,p_true.N-1):
        for k in range(j+1, p_true.N):
            print(f"Correlation between {i},{j},{k}\n",p_true.expectation(lambda s: s[0]*s[1]*s[2], [i,j,k], p_true.p))

Correlation between 0,1,2
 9.004156606172887e-06
Correlation between 0,1,3
 5.097922331429626e-08
Correlation between 0,2,3
 1.7118079779222967e-08
Correlation between 1,2,3
 1.4737371445570076e-06


The main results in the paper are, in the perturbative regime:
- $\Delta_N = D_{K L}\left(p_{\text {true }} \| p_{\text {pair }}\right) / D_{K L}\left(p_{\text {true }} \| p_{\text {ind }}\right)$ scales as $(N-2)\delta + \mathcal{O}\left((N \delta)^{2}\right)$ (I assume the power of 2 is just a safe upper bound). 
- $$D_{K L}\left(p_{\text {true }} \| p_{\text {pair }}\right)=\frac{1}{\ln 2} \sum_{i<j<k} \bar{r}_{i} \bar{r}_{j} \bar{r}_{k} f\left(\bar{\boldsymbol{\rho}}_{ijk}^{\text{true}}, \bar{\boldsymbol{\rho}}_{i j k}^{\text{pair}}\right)+\mathcal{O}\left((N \delta)^{4}\right)$$
- $$D_{K L}\left(p_{\text {true }} \| p_{\text {ind }}\right)=\frac{1}{\ln 2} \sum_{i<j} \bar{r}_{i} \bar{r}_{j} f\left(\rho_{i j}^{\text{true}}, 0\right)+\mathcal{O}\left((N \delta)^{3}\right)$$

where $f(x,y) \doteq (1+x)\big[\ln(1+x)-\ln(1+y)\big] - (x-y)$, $\overline{r}_i = \langle r \rangle $ (they use $r$ instead of $\sigma$ in the paper), $\rho_{ij}$ is the normalised pairwise correlation defined as:
$$
    \rho_{ij} \doteq \frac{\langle r_i r_j \rangle - \bar{r_i} \bar{r_j}}{\bar{r_i} \bar{r_j}}
$$
and $\bar{\boldsymbol{\rho}}_{i j k}^{p}$ is defined as:
$$
    \bar{\boldsymbol{\rho}}_{i j k}^{p} \doteq \frac{\langle r_i r_j r_k \rangle - \bar{r_i} \bar{r_j} \bar{r_k}}{\bar{r_i} \bar{r_j} \bar{r_k}}
$$
Note, $\bar{\boldsymbol{\rho}}_{i j k}^{p}$ depends on the distribution $p$ that is used to calculate the expectations.

The KL divergence is defined as $D_{KL}(p\| q) \doteq \sum_s p_s \ln (p_s/q_s) $, where we sum over all states $s$, which in this case is defined as all configurations of spins. 

In [40]:
def get_statistics(p):
    avgs = p.averages() #get the averages \bar{r}
    corrs_2 = p.correlations() #get the pairwise correlations <r_i r_j>
    
    N = p.N #how many neurons
    
    #calc all the \rho_{ij}
    rho = np.zeros((N,N))
    for i in range(N-1):
        for j in range(i+1,N):
            rho[i,j]= corrs_2[i,j]/(avgs[i]*avgs[j]) - 1
            
    #calc all the \bar{\rho}_{ijk}
    rho_b = np.zeros((N,N,N))
    corrs_3 = np.zeros((N,N,N))
    for i in range(N-2):
        for j in range(i+1,N-1):
            for k in range(j+1,N):
                corrs_3[i,j,k]= p.expectation(lambda s: s[0]*s[1]*s[2], [i,j,k], p.p) 
                rho_b[i,j,k] = corrs_3[i,j,k] / (avgs[i]*avgs[j]*avgs[k]) - 1
                
    return avgs, corrs_2, corrs_3, rho, rho_b

To test all of this out, let's define a model that models the probability of each neuron firing as being independent. This corresponds to setting the 'local fields' $h_i$ to $\ln(\bar{r}_i/(1-\bar{r}_i))$, and all other interaction parameters to 0. The expectations $\langle r_i r_j ... r_p \rangle$ should purely be products of the expectations of seeing indivdual neurons fire $\bar{r}_i \bar{r}_j ... \bar{r}_p$. Hence, the normalised correlations should all be 0.

In [41]:
N = 4
r_bar = np.ones(N)*0.5
h = -np.log(1/r_bar - 1)
J = np.zeros((N,N))
K = np.zeros((N,N,N))

p_ind = ThreeWise(N,h,J,K)

In [42]:
get_statistics(p_ind)

(array([0.5, 0.5, 0.5, 0.5]),
 array([[0.5 , 0.25, 0.25, 0.25],
        [0.  , 0.5 , 0.25, 0.25],
        [0.  , 0.  , 0.5 , 0.25],
        [0.  , 0.  , 0.  , 0.5 ]]),
 array([[[0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.125, 0.125],
         [0.   , 0.   , 0.   , 0.125],
         [0.   , 0.   , 0.   , 0.   ]],
 
        [[0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.125],
         [0.   , 0.   , 0.   , 0.   ]],
 
        [[0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ]],
 
        [[0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ],
         [0.   , 0.   , 0.   , 0.   ]]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         

## Modelling toy distributions with the pairwise model

We consider some toy distributions, one that has all neurons firing or not ("All or nothing model"), another which only takes on 2 states ("A couple of states or nothing model") and a third where one of the neurons only fires if only one of the other 2 neurons fire ("XOR").

### All or nothing model
We start by considering a very simple distribution that has higher order correlations. Let us say we have 5 neurons which always fire in sync. Thus $p(1,1,1,1,1) = c$, $p(0,0,0,0,0)=(1-c)$ and all other events have probability 0. The mean firing rate of individual neurons will be c, as will the correlations. We will vary the probability $c$ of all of them firing, and see whether the pairwise model appears to be a good fit. 

In [2]:
N = 5
c = 0.5
avgs = c*np.ones(N) # prob of every neuron firing in a window is 0.5
corrs = c*np.triu(np.ones((N,N)),1) # prob of 2 neurons firing in the same window is 0.2 
print(avgs,corrs, sep="\n")

[0.5 0.5 0.5 0.5 0.5]
[[0.  0.5 0.5 0.5 0.5]
 [0.  0.  0.5 0.5 0.5]
 [0.  0.  0.  0.5 0.5]
 [0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0. ]]


In [3]:
p_wise = Ising(N, avgs, corrs, lr=0.5) 

In [4]:
p_wise.gradient_ascent() # 100 steps of gradient ascent. Repeat until accurate. 

In [5]:
print("Predicted averages:", p_wise.averages(), "Predicted correlations:", p_wise.correlations(),sep="\n")

Predicted averages:
[0.53619501 0.53738947 0.54056568 0.54641463 0.55341676]
Predicted correlations:
[[0.53619501 0.51575932 0.51198993 0.50910874 0.50459316]
 [0.         0.53738947 0.505844   0.502652   0.49986001]
 [0.         0.         0.54056568 0.50027544 0.49694551]
 [0.         0.         0.         0.54641463 0.49396977]
 [0.         0.         0.         0.         0.55341676]]


Now that we have trained a maximum entropy model, let us see what it thinks the true probability distribution looks like.

In [6]:
for state in [p_wise.states[0],p_wise.states[-1]]:
    print(state,np.round(p_wise.p(state),2))

[0. 0. 0. 0. 0.] 0.37
[1. 1. 1. 1. 1.] 0.46


Interestingly, the pairwise model is able to accurately predict the full probability distribution of the 'all or nothing model' for different values of $c$. I honestly wasn't sure what to expect here, and would be interested in relating this to the results from the Roudi et al. paper. We will have to consider slightly more complex distributions to 'break' the pairwise model. 

## A couple of states or nothing
The next model that came to mind that takes on two states:

- $p(0,1,1,1,1)=a$
- $p(1,1,1,1,0)=b$
- $p(0,0,0,0,0)=1-(a+b)$

We define $a+b \doteq c$

The expectation of the neurons will be: 

    (b, c, c, c, c, a)

The pairwise correlations will be:

        1 2 3 4 5
      1   b b b 0
      2     c c a    
      3       c a
      4         a

In [74]:
N = 5
a = 0.2
b = 0.4
c = a + b
avgs = np.array([b, c, c, c, a])
corrs = np.array([[0,b,b,b,0],
                  [0,0,c,c,a],
                  [0,0,0,c,a],
                  [0,0,0,0,a]])
print(avgs,corrs, sep="\n")

[0.4 0.6 0.6 0.6 0.2]
[[0.  0.4 0.4 0.4 0. ]
 [0.  0.  0.6 0.6 0.2]
 [0.  0.  0.  0.6 0.2]
 [0.  0.  0.  0.  0.2]]


In [75]:
p_wise = Ising(N, avgs, corrs, lr=0.5) 

In [100]:
p_wise.gradient_ascent() # 100 steps of gradient ascent. Repeat until accurate. 

In [101]:
print("Predicted averages:", p_wise.averages(), "Predicted correlations:", p_wise.correlations(),sep="\n")

Predicted averages:
[0.40120964 0.60166855 0.60201396 0.60243361 0.20180175]
Predicted correlations:
[[0.40120964 0.40017571 0.40004125 0.39989105 0.00277631]
 [0.         0.60166855 0.60019171 0.59988656 0.19942072]
 [0.         0.         0.60201396 0.59956424 0.19928799]
 [0.         0.         0.         0.60243361 0.19916612]
 [0.         0.         0.         0.         0.20180175]]


In [103]:
for state in p_wise.states:
    print(state,np.round(p_wise.p(state),3))

[0. 0. 0. 0. 0.] 0.392
[0. 0. 0. 0. 1.] 0.002
[0. 0. 0. 1. 0.] 0.002
[0. 0. 0. 1. 1.] 0.0
[0. 0. 1. 0. 0.] 0.001
[0. 0. 1. 0. 1.] 0.0
[0. 0. 1. 1. 0.] 0.0
[0. 0. 1. 1. 1.] 0.0
[0. 1. 0. 0. 0.] 0.0
[0. 1. 0. 0. 1.] 0.0
[0. 1. 0. 1. 0.] 0.0
[0. 1. 0. 1. 1.] 0.0
[0. 1. 1. 0. 0.] 0.0
[0. 1. 1. 0. 1.] 0.001
[0. 1. 1. 1. 0.] 0.004
[0. 1. 1. 1. 1.] 0.196
[1. 0. 0. 0. 0.] 0.001
[1. 0. 0. 0. 1.] 0.0
[1. 0. 0. 1. 0.] 0.0
[1. 0. 0. 1. 1.] 0.0
[1. 0. 1. 0. 0.] 0.0
[1. 0. 1. 0. 1.] 0.0
[1. 0. 1. 1. 0.] 0.0
[1. 0. 1. 1. 1.] 0.0
[1. 1. 0. 0. 0.] 0.0
[1. 1. 0. 0. 1.] 0.0
[1. 1. 0. 1. 0.] 0.0
[1. 1. 0. 1. 1.] 0.0
[1. 1. 1. 0. 0.] 0.001
[1. 1. 1. 0. 1.] 0.0
[1. 1. 1. 1. 0.] 0.396
[1. 1. 1. 1. 1.] 0.003


Again, the pairwise model is able to capture the probability distribution.  

## XOR 

In the 2003 Schneidman paper, *Network Information and Connected Correlations*, they say,

> If $\sigma_3$ is formed as the exclusive OR (XOR) of the variables $\sigma_1$ and $\sigma_2$, then the essential structure of $p(\sigma_1,\sigma_2,\sigma_3)$ is contained in a three–spin interaction. 

This might give us a simple example of something the ising model can't model.

Let us say that $\sigma_1$ and $\sigma_2$ firing independently with probabilities $p(\sigma_1{=}1)=a$ and $p(\sigma_2{=}1)=b$. 

        s_1 s_2 s_3  p(s_1, s_2, s_3)
        0   0   0    (1-a)(1-b)
        0   1   1    (1-a)b
        1   0   1    a(1-b)
        1   1   0    ab
Thus, the averages are:

        (a, b, b+a-2ab)

And the correlations are:

        s_1 s_2, s_1 s_3, s_2,s_3
        ab       a(1-b)   (1-a)b        

In [2]:
N = 3
a = 0.2
b = 0.4
avgs = np.array([a,b,b+a-2*a*b])
corrs = np.array([[0,a*b,a*(1-b)],
                  [0,0,(1-a)*b]])
print(avgs,corrs, sep="\n")

[0.2  0.4  0.44]
[[0.   0.08 0.12]
 [0.   0.   0.32]]


In [3]:
p_wise = Ising(3, avgs, corrs, lr=0.5) 

In [18]:
p_wise.gradient_ascent() # 100 steps of gradient ascent. Repeat until accurate. 

In [19]:
print("Predicted averages:", p_wise.averages(), "Predicted correlations:", p_wise.correlations(),sep="\n")

Predicted averages:
[0.20000128 0.40000042 0.44000421]
Predicted correlations:
[[0.20000128 0.08000851 0.11999224]
 [0.         0.40000042 0.31999632]
 [0.         0.         0.44000421]]


In [20]:
for state in p_wise.states:
    print(state,np.round(p_wise.p(state),3))

[0. 0. 0.] 0.406
[0. 0. 1.] 0.074
[0. 1. 0.] 0.074
[0. 1. 1.] 0.246
[1. 0. 0.] 0.074
[1. 0. 1.] 0.046
[1. 1. 0.] 0.006
[1. 1. 1.] 0.074


In [22]:
print([0,0,0], (1-a)*(1-b))
print([0,1,1], (1-a)*b)
print([1,0,1], a*(1-b))
print([1,1,0], a*b)

[0, 0, 0] 0.48
[0, 1, 1] 0.32000000000000006
[1, 0, 1] 0.12
[1, 1, 0] 0.08000000000000002


Notice how the events `[0,0,1]` and `[1,1,1]` are assigned non-zero probabilities, when they should in fact be zero. In general, we can see the predictions are far off. 